In [ ]:
# The order of this notebook has been arranged to simulate user modification and out-of-order re-execution.
# Simply run the notebook top-down will be sufficient to demonstrate the steps in task_1 and task_2.
import apache_beam as beam
from apache_beam.runners.interactive import interactive_runner
from apache_beam.runners.interactive.interactive_beam import *

In [ ]:
p = beam.Pipeline(interactive_runner.InteractiveRunner())
init_pcoll = p | beam.Create(range(10))

In [ ]:
squares = init_pcoll | 'Square' >> beam.Map(lambda x: x**x)

In [ ]:
cubes = init_pcoll | 'Cube' >> beam.Map(lambda x: x**3)

In [ ]:
show(squares)

In [ ]:
# In cell 6, fix the typo in cell 3.
squares = init_pcoll | 'Square' >> beam.Map(lambda x: x*x)

In [ ]:
show(squares)

In [ ]:
show(cubes)

In [ ]:
show(cubes)

In [ ]:
p.run()